In [ ]:
import matplotlib.pyplot as plt
import pyart
from pyart.testing import get_test_data

# read in the data from both XSAPR radars
xsapr_sw_file = get_test_data("swx_20120520_0641.nc")
xsapr_se_file = get_test_data("sex_20120520_0641.nc")
radar_sw = pyart.io.read_cfradial(xsapr_sw_file)
radar_se = pyart.io.read_cfradial(xsapr_se_file)

# filter out gates with reflectivity > 100 from both radars
gatefilter_se = pyart.filters.GateFilter(radar_se)
gatefilter_se.exclude_transition()
gatefilter_se.exclude_above("corrected_reflectivity_horizontal", 100)
gatefilter_sw = pyart.filters.GateFilter(radar_sw)
gatefilter_sw.exclude_transition()
gatefilter_sw.exclude_above("corrected_reflectivity_horizontal", 100)

# perform Cartesian mapping, limit to the reflectivity field.
grid = pyart.map.grid_from_radars(
    (radar_se, radar_sw),
    gatefilters=(gatefilter_se, gatefilter_sw),
    grid_shape=(20, 181, 181),
    grid_limits=((500, 10000), (-50000, 40000), (-60000, 40000)),
    grid_origin=(36.57861, -97.363611),
    fields=["corrected_reflectivity_horizontal"],
)

In [ ]:
90_000 / 500

In [ ]:
ds = grid.to_xarray()
ds.z

In [ ]:
ds.y

In [ ]:
start = (36.7, -97.7)
end = (36.2, -97.8)

fig = plt.figure(figsize=(18, 6))
ax1 = plt.subplot(121, projection=ccrs.PlateCarree())
display = pyart.graph.GridMapDisplay(grid)
display.plot_grid(
    "corrected_reflectivity_horizontal",
    ax=ax1,
    cmap="pyart_HomeyerRainbow",
    vmin=-20,
    vmax=70,
)
ax1.scatter(start[1], start[0], color="tab:blue", label="Start")
ax1.scatter(end[1], end[0], color="black", label="End")
ax1.plot([start[1], end[1]], [start[0], end[0]], color="k", linestyle=":")
plt.legend(loc="upper right")

ax2 = plt.subplot(122)
display.plot_cross_section(
    "corrected_reflectivity_horizontal",
    start,
    end,
    x_axis="lat",
    cmap="pyart_HomeyerRainbow",
    vmin=-20,
    vmax=70,
)

In [ ]:
fig = plt.figure()
start = (34.8, -98.75)
end = (38.6, -96.45)
grid = pyart.testing.make_target_grid()
display = pyart.graph.GridMapDisplay(grid)
display.plot_cross_section("reflectivity", start, end, x_axis="lat", vmin=-5, vmax=35)

In [ ]:
display = pyart.graph.GridMapDisplay(grid)
display.plot_cross_section("reflectivity", start, end, x_axis="lat", vmin=-5, vmax=35)

In [ ]:
start = (36.7, -97.7)
end = (36.2, -97.8)


display = pyart.graph.GridMapDisplay(grid)

In [ ]:
list(grid.fields)

In [ ]:
display.plot_cross_section(
    "reflectivity",
    start,
    end,
    x_axis="lon",
    cmap="pyart_HomeyerRainbow",
    vmin=-20,
    vmax=70,
)

In [ ]:
ds = grid.to_xarray()

In [ ]:
params = grid.get_projparams()

In [ ]:
from metpy.interpolate import cross_section

In [ ]:
import cartopy.crs as ccrs

# ds.attrs["projection"] = ccrs.PlateCarree().proj4_params
ds = ds.metpy.parse_cf()

In [ ]:
radar_crs = ccrs.AzimuthalEquidistant(
    central_longitude=params["lon_0"], central_latitude=params["lat_0"]
)

In [ ]:
projection_info = radar_crs.to_cf()

In [ ]:
projection_info

In [ ]:
ds = ds.metpy.assign_crs(projection_info)

In [ ]:
ds["z"] = ds.z / 1000

In [ ]:
ds.z.attrs["units"] = "Distance above radar (km)"

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
ds.corrected_reflectivity_horizontal.isel(z=0).plot(
    x="lon",
    y="lat",
    cmap="pyart_HomeyerRainbow",
    vmin=-20,
    vmax=60,
    ax=ax,
    add_colorbar=False,
)

In [ ]:
start = (36.7, -97.7)
end = (36.2, -97.8)

fig = plt.figure()
ax = plt.subplot(111)
ds.corrected_reflectivity_horizontal.isel(z=0).plot(
    x="lon", y="lat", cmap="pyart_HomeyerRainbow", vmin=-20, vmax=60, ax=ax
)
ax.scatter(start[1], start[0], label="Start Point", color="tab:blue")
ax.scatter(end[1], end[0], label="Start Point", color="k")
ax.plot([start[1], end[1]], [start[0], end[0]], color="k", linestyle=":")
plt.legend()

In [ ]:
cross = cross_section(ds, start, end).set_coords(("lat", "lon"))

In [ ]:
fig = plt.figure(figsize=(10, 8))
title = "($^\circ$N)"
cross.corrected_reflectivity_horizontal.plot(
    cmap="pyart_HomeyerRainbow",
    x=None,
    y="z",
    vmin=-20,
    vmax=60,
)
plt.title(title)

In [ ]:
from pyart.graph.common import generate_grid_time_begin, generate_field_name


def generate_cross_section_title(grid, field, start, end):
    """
    Generate a title for a plot.

    Parameters
    ----------
    grid : Grid
        Radar structure.
    field : str
        Field plotted.
    start : tuple
        Latitude-Longitude pair of starting points
    end: tuple
        Latitude-Longitude pair of ending poins

    Returns
    -------
    title : str
        Plot title.

    """

    # Grab information about the grid
    time_str = generate_grid_time_begin(grid).strftime("%Y-%m-%dT%H:%M:%SZ")
    field_name = generate_field_name(grid, field)

    # Deal with the latitude/longitude part
    start_lat, start_lon = start
    end_lat, end_lon = end

    degree_symbol = "\N{DEGREE SIGN}"

    # Add degrees latitude/longitude labels

    if start_lat < 0:
        start_lat_string = f"{-start_lat}{degree_symbol}S"
    else:
        start_lat_string = f"{start_lat}{degree_symbol}N"

    if end_lat < 0:
        end_lat_string = f"{-end_lat}{degree_symbol}S"
    else:
        end_lat_string = f"{end_lat}{degree_symbol}N"

    if start_lon < 0:
        start_lon_string = f"{-start_lon}{degree_symbol}W"
    else:
        start_lon_string = f"{start_lon}{degree_symbol}E"

    if end_lon < 0:
        end_lon_string = f"{-end_lon}{degree_symbol}W"
    else:
        end_lon_string = f"{end_lon}{degree_symbol}E"

    return (
        f"({start_lat_string}, {start_lon_string})"
        f" to ({end_lat_string}, {end_lon_string}) \n {time_str} \n {field_name}"
    )

In [ ]:
fig = plt.figure(figsize=(10, 8))
title = generate_cross_section_title(
    grid, "corrected_reflectivity_horizontal", start, end
)
cross.corrected_reflectivity_horizontal.plot(
    cmap="pyart_HomeyerRainbow",
    x=None,
    y="z",
    vmin=-20,
    vmax=60,
)
plt.title(title)

In [ ]:
title